Codigo para obtener los datos de vuelo de starship de los videos de los lanzamientos

Obtener la posicion del raton en pantalla para obtener los rectangulos a capturar. Es util para obtener las posiciones de la pantalla a capturar

Tambien se puede hacer una captura de pantalla completa con paint y poner los valores directamente

In [14]:
import pyautogui
import time

try:
    while True:
        # Obtiene la posición del ratón
        x, y = pyautogui.position()
        # Imprime la posición
        print(f'Posición del ratón: ({x}, {y})')
        # Espera un poco antes de obtener la siguiente posición
        time.sleep(1)
except KeyboardInterrupt:
    print('Programa terminado')

Posición del ratón: (-984, 265)
Posición del ratón: (1220, 836)
Posición del ratón: (1190, 833)
Posición del ratón: (1189, 829)
Posición del ratón: (1189, 829)
Posición del ratón: (1189, 829)
Posición del ratón: (1189, 829)
Posición del ratón: (1189, 829)
Posición del ratón: (1189, 829)
Posición del ratón: (1235, 843)
Posición del ratón: (1234, 846)
Posición del ratón: (1235, 849)
Posición del ratón: (1235, 849)
Posición del ratón: (1235, 849)
Posición del ratón: (1235, 849)
Posición del ratón: (1235, 849)
Posición del ratón: (1235, 849)
Posición del ratón: (-803, 324)
Programa terminado


In [1]:
# Definir las regiones de los rectángulos (left, top, width, height)
#La funcion rectangulo te calcula el rectangulo dados la 
# esquina superior izquierda y la esquina inferior derecha

def rectangulo(a, b, c, d):
    return (a, b, c - a, d - b)

rectangles = [
    ('Hora', rectangulo(708, 829,742, 855)),
    ('Minuto', rectangulo(748, 830,782, 854)),
    ('Segundo', rectangulo(787, 829,821, 855)),
    ('Velocidad Booster', rectangulo(313, 802, 376, 822)),
    ('Altura Booster', rectangulo(331, 829,376, 848)),
    ('Velocidad Starship', rectangulo(1147, 804,1233, 823)),
    ('Altura Starhip', rectangulo(1189, 829,1235, 849))
]

Script para obtener los datos de vuelo

In [2]:
#Importamos librerias
import pyautogui
import cv2
import pandas as pd
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import easyocr

In [3]:
reader = easyocr.Reader(['en'])

def capture_region(region):
    screenshot = pyautogui.screenshot(region=region)
    return cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)

def extract_text_from_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    results = reader.readtext(gray, detail=0)
    text = ' '.join(results)
    return text.strip()

def process_region(name, region):
    image = capture_region(region)
    text = extract_text_from_image(image)
    return name, text

results = []

id = 0

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [4]:
#Si se quiere hacer hasta pararlo tu manualmente
while True:
    row = {'id': id}
    with ThreadPoolExecutor(max_workers=len(rectangles)) as executor:
        futures = {executor.submit(process_region, name, region): name for name, region in rectangles}
        for future in as_completed(futures):
            name, text = future.result()
            row[name] = text
    # print(row)
    results.append(row)
    id = id + 1

KeyboardInterrupt: 

In [5]:
#Si se quiere hacer hasta x veces

numeroDeVeces = 10

for _ in range(10):
    row = {'id': id}
    with ThreadPoolExecutor(max_workers=len(rectangles)) as executor:
        futures = {executor.submit(process_region, name, region): name for name, region in rectangles}
        for future in as_completed(futures):
            name, text = future.result()
            row[name] = text
    results.append(row)
    id = id + 1

In [5]:
#Guardamos datos
df = pd.DataFrame(results)
df.to_csv('output.csv', index=False)

print("Datos guardados en output.csv")

Datos guardados en output.csv
